# Zero Inflated Model

During the project, I began exploring a zero-inflated, two-step modeling process. As I describe in the report, due to sparsity and lack of nonzero target samples, I ended up giving up on this approach. However, I included it in the code because I spent quite a bit of time setting this up and it has some interesting results.

### Import the Data

In [6]:
import pandas as pd
from metrics import *
from dataloaders import *
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression, PoissonRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, zero_one_loss
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [20]:
# multiomics_clean = pd.read_csv("../data_csv/multiomics_svd_biplot_1_3_21.csv")
# multiomics_clean = pd.read_csv("../data_csv/multiomics_svd_100x100.csv")
# multiomics_clean = pd.read_csv("../data_csv/multiomics_auc_500.csv")
multiomics_clean = pd.read_csv("../data_csv/multiomics_best_slides.csv")


target_tags = pd.Index(['2D10-LTR5', '2D10-Tat', '2D10-Vpu', '2D10-Env', '2D10-d2EGFP',
       '2D10-LTR3', 'Target'])
other_columns = multiomics_clean.columns.drop(target_tags).drop(pd.Index(["dataset"]))

pd_train = multiomics_clean.loc[multiomics_clean["dataset"] == "train"]
pd_test = multiomics_clean.loc[multiomics_clean["dataset"] == "test"]
pd_total = multiomics_clean

y_train = (pd_train["Target"].to_numpy() > 0).astype(int)
X_train = pd_train[other_columns].to_numpy()
y_test = (pd_test["Target"].to_numpy() > 0).astype(int)
X_test = pd_test[other_columns].to_numpy()


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train)

(3121, 18)
(3121,)
(779, 18)
(779,)
[0 0 1 ... 0 0 0]


# The ZI Model: Logistic Regression > Poisson Regression
First, I predict the binary case using a logistic regressor, then I estimate the counts using a poisson regressor. This one does not work on the full target because Poisson regression directly estimates integer count values.

Note that I have to include an if statement to see whether task 1 even predicts anything from the test set as nonzero. It oftentimes due to class imbalance will just predict all zeros, leaving no data for task 2 and breaking the pipeline.

My solution to this is training task 2 on all elements of the test set that have nonzero ground truth labels rather than using the predictions of task 1, but even then, the training set size for task 2 is 4-20 samples which is insufficient to train a model on especially considering the feature space.

In [ ]:
kf = StratifiedKFold(n_splits = 5)

hiv_counts = target_tags.delete(-1)
print(hiv_counts)

X = pd_total[other_columns].to_numpy()
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Loop over the 6 HIV genes
for target_tag in target_tags:
    
    y = (pd_total[target_tag].to_numpy()).astype(int)
    
    aucs = []
    aucmax = 0.0
    best_probs = np.array([])
    best_test = np.array([])
    
    # Loop over the folds
    for splitno, indices in enumerate(kf.split(X, (y>0).astype(int))):
        train_index, test_index = indices
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_train_zero = (y_train > 0).astype(int)
        y_test_zero = (y_test > 0).astype(int)
    
        # Use a logistic regressor to compute task 1
        clf = LogisticRegression(random_state=0, solver='liblinear')
        clf.fit(X_train, y_train_zero)
        print("Logistic Score: {}".format(clf.score(X_test, y_test_zero)))

        y_probs = clf.predict_proba(X_test)
        y_preds = clf.predict(X_test)
        
        phis = y_probs
        
        
        X_train_nonzero = X_train[y_train != 0, :]
        y_train_nonzero = y_train[y_train != 0]
        X_test_nonzero = X_test[clf.predict(X_test) == 1, :]
        y_test_nonzero = y_test[clf.predict(X_test) == 1]
        
        # Note the if statement here. Oftentimes the y_test_nonzero set (which the poisson regressor is trained on) 
        # would have no values in it if the first task was inaccurate and failed to find any HIV genes in the test set.
        # This illustrates the issue of not having enough data to make this two stage process function well.
        if len(y_test_nonzero) > 0:
            clfpois = PoissonRegressor()
            clfpois.fit(X_train_nonzero, y_train_nonzero)
#             print(clfpois.predict(X_test_nonzero))
            print("Pois Nonzero Score: {}".format(clfpois.score(X_test_nonzero, y_test_nonzero)))
        
            y_pred_nonzero = clfpois.predict(X_test_nonzero)
            y_preds = np.where(clf.predict(X_test) == 1, clfpois.predict(X_test), clf.predict(X_test))
        else:
            y_preds = clf.predict(X_test)

        auc = binary_metrics(y_test_zero, y_probs[:, 1], binary=False, title="R.Forest, Gene {}, Split {}".format(target_tag, splitno))
        aucs.append(auc)
        if np.absolute(auc-0.5) > aucmax:
            best_probs = y_probs[:, 1]
            best_test = y_test_zero
            aucmax = np.absolute(auc-0.5)
    print("AUC mean: {}".format(np.mean(aucs)))
    binary_metrics(best_test, best_probs, binary=False,  title="R.Forest, Gene {}".format(target_tag))

# (DEPRECATED) Attempt at out of the box ZIP model

This out of the box ZIP model failed to work at all, I only include it here as evidence of my work.

In [ ]:
for target_tag in target_tags:
    X = pd_total[other_columns].to_numpy()
    y = (pd_total[target_tag].to_numpy()).astype(int)
    
    for splitno, indices in enumerate(tqdm(kf.split(X, (y>0).astype(int)))):
        train_index, test_index = indices
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, inflation="logit")
        zip_training_results = clf.fit(maxiter=100)
        
        print(zip_training_results.summary())

# Random Forest Classifier > Random Forest Regressor

Finally, I try using a regressor in the second stage of my task rather than a classifier. Here, my results are again unimpressive. Moving fowards, I will focus on improving the fidelity of the binary clasification case because of a lack of data for task 2.

In [32]:
kf = StratifiedKFold(n_splits = 5)



for target_tag in target_tags:
    X = pd_total[other_columns].to_numpy()
    y = (pd_total[target_tag].to_numpy()).astype(int)
    
    bestmse = 100000
    for splitno, indices in enumerate(kf.split(X, (y>0).astype(int))):
        train_index, test_index = indices
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_train_zero = (y_train > 0).astype(int)
        y_test_zero = (y_test > 0).astype(int)
        
        
    
#         y_train = (pd_train[target_tag].to_numpy() > 0).astype(int)
#         X_train = pd_train[other_columns].to_numpy()
#         y_test = (pd_test[target_tag].to_numpy() > 0).astype(int)
#         X_test = pd_test[other_columns].to_numpy()


        clf = RandomForestClassifier(bootstrap=True, random_state=0, verbose=False)
        clf.fit(X_train, y_train_zero)
        print("CLF Score: {}".format(clf.score(X_test, y_test)))

        y_probs = clf.predict_proba(X_test)
        y_preds = clf.predict(X_test)
        
        X_train_nonzero = X_train[clf.predict(X_train) == 1, :]
        y_train_nonzero = y_train[clf.predict(X_train) == 1]
        X_test_nonzero = X_test[clf.predict(X_test) == 1, :]
        y_test_nonzero = y_test[clf.predict(X_test) == 1]
        print(y_train_nonzero.shape)
        if len(y_train_nonzero) > 0:
        
            clfpois = RandomForestRegressor()
            clfpois.fit(X_train_nonzero, y_train_nonzero)
            
            if len(y_test_nonzero) > 0:
                print("Regressor Score: {}".format(clfpois.score(X_test_nonzero, y_test_nonzero)))

                y_pred_nonzero = clfpois.predict(X_test_nonzero)
            y_preds = np.where(clf.predict(X_test) == 1, clfpois.predict(X_test), clf.predict(X_test))

            print("MSE: {}, Target: {}, Split: {}".format(mean_squared_error(y_preds, y_test), target_tag, splitno))
            if mean_squared_error(y_preds, y_test) < bestmse:
                bestmse = mean_squared_error(y_preds, y_test)
                bestypreds = y_preds
                bestytest = y_test
        
    print("Best MSE: {}, Target: {}".format(mean_squared_error(bestypreds, bestytest), target_tag))
#     print(bestypreds)
#     print(bestytest)


#         binary_metrics(y_test_zero, y_probs[:, 1], binary=False, title="ZIP, Gene {}, Split {}".format(target_tag, splitno))
#         binary_metrics(y_test_zero, y_preds, title="ZIP Full", binary=True)

CLF Score: 0.9564102564102565
(136,)
MSE: 0.1794871794871795, Target: 2D10-LTR5, Split: 0
CLF Score: 0.9551282051282052
(136,)


C:\Users\alexo\Anaconda3\envs\cs474\lib\site-packages\sklearn\metrics\_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Pois Nonzero Score: nan
MSE: 0.309900641025641, Target: 2D10-LTR5, Split: 1
CLF Score: 0.9564102564102565
(136,)
MSE: 0.11666666666666667, Target: 2D10-LTR5, Split: 2
CLF Score: 0.9564102564102565
(136,)
MSE: 0.06666666666666667, Target: 2D10-LTR5, Split: 3
CLF Score: 0.9564102564102565
(136,)
MSE: 0.6628205128205128, Target: 2D10-LTR5, Split: 4
Best MSE: 0.06666666666666667, Target: 2D10-LTR5
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   10.51]
[ 3  2  4 15  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  6  0  0  0  0
 12  0  0  3  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 11  0
  0  1  0  0  0  0  0  0  0  0  7  0  2  0  7  5  0  0  4  0  0  0  0  0
  0  0  0  0  3  0  0  0  0  0  0  0  0  0  6  0  8  0  0  1  0  0  0  0
  1  0  0  0  2  1  0  0  0  0  0  1  0  0  0  0  0  0  0 13  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  4  0  0  0  0  0  0  2  0  0  0  4  0  0
  0  0  0  0  0  1  1  0  0  7  0 15  0  6  0  0  0  0  0  0  1  1  0  0
  0  0  0  0  0  0  2  0  0  0  0 22  0  4  0  0  0  0  0 25  0  0  0  0
  0  0  0 12  0  0  0  0  1  1  0  0  0  0 34  0  0  0  0  0  0  0 45  0
  0  0  0  1  0  0  1  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0
  0  0  0  0  0  0  1  0  1  0  0  0  1  0  0  8  1  0 18  0  0  0  0  0
  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  1  0  0 26  0  0  0  0  2
  0  0  0  0  0  2  0  0  5 21  1  0  0  0  3  5  

C:\Users\alexo\Anaconda3\envs\cs474\lib\site-packages\sklearn\metrics\_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


...

C:\Users\alexo\Anaconda3\envs\cs474\lib\site-packages\sklearn\metrics\_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Pois Nonzero Score: nan
MSE: 0.1092948717948718, Target: 2D10-d2EGFP, Split: 0
CLF Score: 0.9564102564102565
(136,)
MSE: 0.09871794871794871, Target: 2D10-d2EGFP, Split: 1
CLF Score: 0.9538461538461539
(136,)
Pois Nonzero Score: 0.0
MSE: 0.07179897435897435, Target: 2D10-d2EGFP, Split: 2
CLF Score: 0.9551282051282052
(136,)


C:\Users\alexo\Anaconda3\envs\cs474\lib\site-packages\sklearn\metrics\_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Pois Nonzero Score: nan
MSE: 0.1042975641025641, Target: 2D10-d2EGFP, Split: 3
CLF Score: 0.9564102564102565
(136,)
MSE: 0.08974358974358974, Target: 2D10-d2EGFP, Split: 4
Best MSE: 0.07179897435897435, Target: 2D10-d2EGFP
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.

CLF Score: 1.0
(0,)
CLF Score: 1.0
(0,)
CLF Score: 1.0
(0,)
CLF Score: 1.0
(0,)
CLF Score: 1.0
(0,)
Best MSE: 16.169409743589743, Target: Target
[ 0.    0.    0.    0.    0.    3.63  0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.   13.19  8.47  0.    0.    0.    0.    0.
  7.21  0.    0.    0.    0.    7.59  0.    0.    0.    9.06  0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    4.18
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    8.97  0.
  0.    4.06  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    7.58